In [1]:
!pip install transformers datasets torch tqdm scikit-learn

In [2]:
# 라이브러리 임포트
import torch
import pandas as pd
import numpy as np
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# KoELECTRA 토크나이저 및 모델 로드
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=4).to(device)  # 4개 클래스 분류


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import ElectraTokenizer

# KoELECTRA 모델 및 토크나이저 로드
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

# 파일 경로
train_file_path = "/kaggle/input/aiffel-dl-thon-dktc-online-12/train.csv"

# 데이터 로드
train_df = pd.read_csv(train_file_path)
more_train_file_path= "/kaggle/input/unmlve/train_normal_friend_couple_family.csv"
more_train_df= pd.read_csv(more_train_file_path)
train_df= pd.concat([train_df, more_train_df], ignore_index=True)

# 라벨 인코딩
label_encoder = LabelEncoder()
train_df["label"] = label_encoder.fit_transform(train_df["class"])

# KoELECTRA 데이터셋 클래스 정의
class KoELECTRADataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding="max_length",  # 🔥 변경: 고정된 패딩 길이
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }


# 데이터셋 생성 (max_length=256 적용)
train_dataset = KoELECTRADataset(
    texts=train_df["conversation"].tolist(),
    labels=train_df["label"].tolist(),
    tokenizer=tokenizer,
    max_length=512,  # 적용됨
)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 데이터셋 크기 확인
print(f"총 데이터 개수: {len(train_dataset)}")

총 데이터 개수: 4550


In [4]:
# pred_label 컬럼의 값 개수 세기
label_counts = train_df['class'].value_counts()

# 결과 출력
print(label_counts)

class
기타 괴롭힘 대화      1094
갈취 대화           981
직장 내 괴롭힘 대화     979
협박 대화           896
일반 대화           600
Name: count, dtype: int64


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import ElectraForSequenceClassification, AdamW
from tqdm import tqdm

# KoELECTRA 모델 로드
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5).to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=3e-5)

# 학습 루프
num_epochs = 15
best_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()

        # 배치 데이터 GPU로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # 모델 출력
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 역전파 & 최적화
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Loss: {avg_loss:.4f}")

    # 모델 저장 (최적 모델)
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), "/kaggle/working/best_model.pt")
        print("✅ Model Saved!")

print("🎉 Training Finished!")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 285/285 [04:04<00:00,  1.16it/s, loss=0.12]  


Epoch 1 Loss: 0.7786
✅ Model Saved!


Epoch 2: 100%|██████████| 285/285 [04:05<00:00,  1.16it/s, loss=0.042] 


Epoch 2 Loss: 0.2440
✅ Model Saved!


Epoch 3: 100%|██████████| 285/285 [04:04<00:00,  1.16it/s, loss=0.0225]


...

Epoch 12 Loss: 0.0406


Epoch 13: 100%|██████████| 285/285 [04:05<00:00,  1.16it/s, loss=0.00295]


Epoch 13 Loss: 0.0260


Epoch 14:  28%|██▊       | 81/285 [01:10<02:57,  1.15it/s, loss=0.0019]  


KeyboardInterrupt: 

In [39]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraForSequenceClassification

# 파일 경로
test_file_path = "/kaggle/input/aiffel-dl-thon-dktc-online-12/test.csv"
submission_file_path = "/kaggle/working/submission.csv"
model_path = "/kaggle/working/best_model.pt"

# 테스트 데이터 로드
test_df = pd.read_csv(test_file_path)

# 모델 로드
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()  # 평가 모드로 설정

# 테스트 데이터셋 클래스 정의
class TestKoELECTRADataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
        }

# 🔥 test_df["conversation"] → test_df["text"]로 수정
test_dataset = TestKoELECTRADataset(
    texts=test_df["text"].tolist(),  # ✅ 여기 수정됨!
    tokenizer=tokenizer,
    max_length=512,
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 예측 수행
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())

# 예측된 클래스 숫자를 원래 라벨(`class`)로 변환
test_df["class"] = label_encoder.inverse_transform(predictions)

# 결과 저장
test_df[["idx", "class"]].to_csv(submission_file_path, index=False)
print(f"✅ 예측 완료! 결과 저장: {submission_file_path}")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-39-1fc9ec158376>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

✅ 예측 완료! 결과 저장: /kaggle/working/submission.csv


In [53]:
import torch
import pandas as pd
import torch.nn.functional as F




# 기존 서브미션 파일 로드
submission_path = "/kaggle/working/submission.csv"
submission_df = pd.read_csv(submission_path)

# 모델 로드
model_path = "/kaggle/working/best_model.pt"
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# 테스트 데이터 로드
test_file_path = "/kaggle/input/aiffel-dl-thon-dktc-online-12/test.csv"
test_df = pd.read_csv(test_file_path)

# 테스트 데이터셋 클래스 정의
class TestKoELECTRADataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
        }

# 데이터 로딩
test_dataset = TestKoELECTRADataset(
    texts=test_df["text"].tolist(),
    tokenizer=tokenizer,
    max_length=512,
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 예측 수행
predictions = []
probabilities = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        probs = F.softmax(outputs.logits, dim=1)  # 확률 변환

        max_probs, preds = torch.max(probs, dim=1)
        predictions.extend(preds.cpu().numpy())
        probabilities.extend(max_probs.cpu().numpy())

# 클래스 매핑
class_mapping = {
    0: "갈취 대화",
    1: "기타 괴롭힘 대화",
    2: "일반 대화",
    3: "직장 내 괴롭힘 대화",
    4: "협박 대화",
}

# Softmax 확률이 0.3 이하인 경우 일반 대화로 자동 보정
for i in range(len(probabilities)):
    if probabilities[i] < 0.9:
        predictions[i] = 2  # 일반 대화

# 최종 결과 저장
test_df["class"] = [class_mapping[p] for p in predictions]
test_df[["idx", "class"]].to_csv("/kaggle/working/submission_adjusted.csv", index=False)

print("✅ 일반 대화 강화 후 서브미션 저장 완료! 🚀")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-53-03f7461e7508>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

✅ 일반 대화 강화 후 서브미션 저장 완료! 🚀


# softmax 기반 처리 전 submission 결과: 0.68678

In [40]:
#softmax 기반 필터링 미적용
# pred_label 컬럼의 값 개수 세기
label_counts = test_df['class'].value_counts()

# 결과 출력
print(label_counts)

class
갈취 대화          156
기타 괴롭힘 대화      134
협박 대화          105
직장 내 괴롭힘 대화     99
일반 대화            6
Name: count, dtype: int64


# softmax 처리 후 submission 결과: 0.69439  
# 후처리 기법 별 영향 없는듯

In [54]:
#softmax 기반 필터링 적용
# pred_label 컬럼의 값 개수 세기
label_counts = test_df['class'].value_counts()

# 결과 출력
print(label_counts)

class
갈취 대화          146
기타 괴롭힘 대화      119
협박 대화           97
직장 내 괴롭힘 대화     92
일반 대화           46
Name: count, dtype: int64


In [ ]:

submission_df = test_df

# 클래스 매핑 정의
class_mapping = {
    "협박 대화": "00",
    "갈취 대화": "01",
    "직장 내 괴롭힘 대화": "02",
    "기타 괴롭힘 대화": "03",
    "일반 대화": "04",
}

# 클래스명 → 숫자로 변환
submission_df["class_no"] = submission_df["class"].map(class_mapping)
submission_df.drop(columns=["class"], inplace=True)
# 컬럼명 변환
submission_df = submission_df.rename(columns={"class_no": "class"})

# 최종 형식 맞추기
submission_df = submission_df[["idx", "class"]]

# 새로운 서브미션 파일 저장
submission_final_path = "submission_final.csv"
submission_df.to_csv(submission_final_path, index=False)

print(f"✅ 서브미션 파일 변환 완료! 저장 경로: {submission_final_path}")

✅ 서브미션 파일 변환 완료! 저장 경로: submission_final.csv


In [27]:
import torch.nn.functional as F
submission_file_path_5 = "/kaggle/working/submission_5.csv"

# 테스트 데이터셋 클래스 정의
class TestKoELECTRADataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=256,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
        }

# 테스트 데이터셋 및 DataLoader 생성
test_dataset = TestKoELECTRADataset(
    texts=test_df["text"].tolist(),
    tokenizer=tokenizer,
    max_length=256,
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 예측 수행
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)  # 확률 변환

        max_probs, preds = torch.max(probs, dim=1)
        predictions.extend(zip(preds.cpu().numpy(), max_probs.cpu().numpy()))

# 예측된 클래스 숫자를 원래 라벨(`class`)로 변환
pred_labels = [label_encoder.inverse_transform([p[0]])[0] for p in predictions]
pred_probs = [p[1] for p in predictions]

# 일반 대화 필터링 기준: 확률이 일정 임계값 이하일 경우
threshold = 0.8  # 확신이 낮은 샘플을 일반 대화로 간주
for i in range(len(pred_probs)):
    if pred_probs[i] < threshold:
        pred_labels[i] = "일반 대화"

# 결과 저장
test_df["class"] = pred_labels
test_df[["idx", "class"]].to_csv(submission_file_path_5, index=False)
print(f"✅ 예측 완료! 일반 대화 필터링 적용됨. 결과 저장: {submission_file_path_5}")


✅ 예측 완료! 일반 대화 필터링 적용됨. 결과 저장: /kaggle/working/submission_5.csv


In [ ]:
import torch
import numpy as np
import torch.nn.functional as F

# 학습 데이터에서 각 클래스의 평균 벡터(μ)와 공분산 행렬(Σ) 구하기
class_means = {}  # 각 클래스별 평균 벡터
class_cov_inv = {}  # 각 클래스별 공분산 행렬의 역행렬

features = {label: [] for label in label_encoder.classes_}  # 클래스별 특징 벡터 저장

with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].cpu().numpy()

        # 모델의 마지막 히든 스테이트 가져오기
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]  # 마지막 히든 레이어
        embeddings = hidden_states.mean(dim=1).cpu().numpy()  # 평균 풀링

        # 각 클래스별 특징 벡터 저장
        for i, label in enumerate(labels):
            features[label_encoder.inverse_transform([label])[0]].append(embeddings[i])

# 각 클래스별 평균 벡터 & 공분산 행렬 계산
for label in features:
    class_means[label] = np.mean(features[label], axis=0)
    cov_matrix = np.cov(np.array(features[label]).T)
    class_cov_inv[label] = np.linalg.pinv(cov_matrix + np.eye(cov_matrix.shape[0]) * 1e-6)  # 안정적 역행렬 계산

In [35]:
# Mahalanobis 거리 계산 함수
def mahalanobis_distance(x, mean, cov_inv):
    delta = x - mean
    return np.sqrt(np.dot(np.dot(delta, cov_inv), delta.T))

# 테스트 데이터에서 Mahalanobis Distance 계산
distances = []
predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]  # 마지막 히든 레이어 벡터
        embeddings = hidden_states.mean(dim=1).cpu().numpy()  # 평균 풀링

        # Mahalanobis 거리 계산
        batch_preds = []
        for emb in embeddings:
            min_dist = float("inf")
            best_class = None
            for class_label in class_means:
                dist = mahalanobis_distance(emb, class_means[class_label], class_cov_inv[class_label])
                if dist < min_dist:
                    min_dist = dist
                    best_class = class_label
            distances.append(min_dist)
            batch_preds.append(best_class)

        predictions.extend(batch_preds)

# Mahalanobis 임계값 설정 (95% 이상 벗어난 데이터는 일반 대화로 분류)
threshold = np.percentile(distances, 85)
for i in range(len(distances)):
    if distances[i] > threshold:
        predictions[i] = "일반 대화"

# 결과 저장
test_df["class"] = predictions
test_df[["idx", "class"]].to_csv("/kaggle/working/submission_mahalanobis.csv", index=False)
print("✅ Mahalanobis Distance 기반 예측 완료! 결과 저장됨.")


✅ Mahalanobis Distance 기반 예측 완료! 결과 저장됨.


In [9]:
from scipy.stats import weibull_min

# 각 클래스별 최고 점수 분포(Weibull Distribution) 계산
weibull_params = {}

for label in features:
    max_logits = []
    with torch.no_grad():
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].cpu().numpy()

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits.cpu().numpy()

            for i, label_idx in enumerate(labels):
                if label_encoder.inverse_transform([label_idx])[0] == label:
                    max_logits.append(np.max(logits[i]))  # 최고 점수 저장

    # Weibull 분포 피팅
    shape, loc, scale = weibull_min.fit(max_logits, floc=0)
    weibull_params[label] = (shape, scale)

# 테스트 데이터에서 OpenMax 적용
openmax_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits.cpu().numpy()

        for i, logit in enumerate(logits):
            max_score = np.max(logit)
            best_class = label_encoder.inverse_transform([np.argmax(logit)])[0]

            # Weibull 분포에서 벗어나면 일반 대화로 분류
            shape, scale = weibull_params[best_class]
            weibull_cdf = weibull_min.cdf(max_score, shape, scale=scale)

            if weibull_cdf < 0.05:  # 5% 확률 이하인 데이터는 일반 대화로 설정
                openmax_preds.append("일반 대화")
            else:
                openmax_preds.append(best_class)

# 결과 저장
test_df["class"] = openmax_preds
test_df[["idx", "class"]].to_csv("/kaggle/working/submission_openmax.csv", index=False)
print("✅ OpenMax 기반 예측 완료! 결과 저장됨.")


OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [ ]:
# pred_label 컬럼의 값 개수 세기
label_counts = test_df['class'].value_counts()

# 결과 출력
print(label_counts)

class
갈취 대화          132
협박 대화          126
기타 괴롭힘 대화      125
직장 내 괴롭힘 대화    113
일반 대화            4
Name: count, dtype: int64


In [11]:
test_df[["idx", "class"]].to_csv("/kaggle/working/submission_openmax.csv", index=False)
print("✅ OpenMax 기반 예측 완료! 결과 저장됨.")


✅ OpenMax 기반 예측 완료! 결과 저장됨.
